In [1]:
import requests
import os
import pandas as pd
import json
import sqlite3

In [29]:
ticker_list = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NVDA', 'META', 'TSLA', 'AVGO', 'ORCL', 'ADBE', 'CSCO', 'CRM', 'NFLX', 'AMD', 'TXN', 'INTC', 'INTU', 'IBM', 'QCOM', 'AMAT', 'BKNG', 'NOW', 'ADP', 'ADI', 'LRCX', 'UBER', 'ABNB', 'FI', 'MU', 'EQIX', 'ATVI', 'PYPL', 'KLAC', 'VMW', 'SNPS', 'PANW', 'CDNS', 'WDAY', 'ANET', 'SNOW', 'MRVL', 'ROP', 'MCHP', 'FTNT', 'ON', 'ADSK', 'TTD', 'IQV', 'DELL', 'SQ', 'PLTR', 'CRWD', 'FIS', 'DDOG', 'EA', 'CSGP', 'DASH', 'GFS', 'HPQ', 'GPN', 'VEEV', 'SYM', 'KEYS', 'MDB', 'ALGN', 'ANSS', 'MPWR', 'HUBS', 'TTWO', 'EBAY', 'HPE', 'NET', 'FICO', 'ZS', 'ZM', 'COIN', 'ENPH', 'SMCI', 'SWKS', 'SNAP', 'SPLK', 'PTC', 'PAYC', 'NTAP', 'FDS', 'EXPE', 'TYL', 'ENTG', 'BSY', 'U', 'AKAM', 'EPAM', 'DT', 'SSNC', 'CHWY', 'TRMB', 'GEN', 'IOT', 'LDOS', 'Z', 'LSCC', 'ZBRA', 'JKHY', 'ROKU', 'AZPN', 'MTCH', 'OKTA', 'MANH', 'PCTY', 'PSTG', 'CDAY', 'TWLO', 'XM', 'GDDY', 'APP', 'TOST', 'DOX', 'DOCU', 'QRVO', 'HOOD', 'ETSY', 'CFLT', 'DBX', 'FFIV', 'W']
len(ticker_list)

125

In [2]:
# Provides better readability when parsing the raw Json files.
pd.set_option('display.max_columns', None)

In [3]:
API_Key = os.environ['POLYGON_API_KEY']

In [23]:
URL = 'https://api.polygon.io/vX/reference/financials'

headers = {
    'Authorization': 'Bearer %s' % API_Key
}
    
params = {
    'ticker': 'SHOP',
    'filing_date.gte': '2020-01-01',
    'timeframe': 'annual',
    'order': 'asc',
    'limit': 100,
    'sort': 'period_of_report_date'
}
    

response = requests.request(
    'GET', URL, params = params, headers = headers
)

print(response.status_code)
print(response.headers)

temp_Json = json.loads(
    response.text
)

200
{'Server': 'nginx/1.19.2', 'Date': 'Mon, 07 Aug 2023 18:31:01 GMT', 'Content-Type': 'application/json', 'Content-Length': '100', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'X-Item-Count': '0', 'X-Request-Id': 'ef20d7cb3d031c11f07acc1e5dab627e', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains'}


In [24]:
temp_Json

{'results': [],
 'status': 'OK',
 'request_id': 'ef20d7cb3d031c11f07acc1e5dab627e'}

In [21]:
temp_DF = pd.json_normalize(
    temp_Json['results'][0], max_level = 1
)
temp_DF.head()

IndexError: list index out of range

In [11]:
file_name = temp_Json['results'][0]['tickers'][0]

with open(f'{file_name}_Annual_Financials.json', 'w') as outfile:
    json.dump(temp_Json, outfile)

In [14]:
with open(f'{file_name}_Annual_Financials.json') as json_file:
    data = json.load(json_file)

In [15]:
data

{'results': [{'id': '0001045810:2020:FY',
   'start_date': '2019-01-28',
   'end_date': '2020-01-26',
   'filing_date': '2020-02-20',
   'timeframe': 'annual',
   'fiscal_period': 'FY',
   'fiscal_year': '2020',
   'cik': '0001045810',
   'sic': '3674',
   'tickers': ['NVDA'],
   'company_name': 'NVIDIA CORP',
   'source_filing_url': 'https://api.polygon.io/v1/reference/sec/filings/0001045810-20-000010',
   'source_filing_file_url': 'http://api.polygon.io/v1/reference/sec/filings/0001045810-20-000010/files/nvda-2020x10k_htm.xml',
   'financials': {'cash_flow_statement': {'net_cash_flow_from_operating_activities': {'value': 4761000000.0,
      'unit': 'USD',
      'label': 'Net Cash Flow From Operating Activities',
      'order': 100},
     'net_cash_flow_from_financing_activities': {'value': -792000000.0,
      'unit': 'USD',
      'label': 'Net Cash Flow From Financing Activities',
      'order': 700},
     'net_cash_flow_from_investing_activities': {'value': 6145000000.0,
      'unit